# 課題3：タイタニック号乗客の生存状況の分類モデル作成

本課題では、`titanic` というデータセットを使います。これは、1912年に発生したタイタニック号の沈没事故における乗客の生存状況に関するデータセットです。元々は、[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/)で掲載されたデータと言われており、このデータセットを組み込んだPythonのライブラリも複数あります。

今回は、`seaborn` のライブラリに組み込まれた `titanic` のデータセットを使います。各セルに入っているコメントの下に、実行するコードを記入してください。わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [1]:
# 必要なライブラリのimport（変更しないでください）
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

# DataFrameですべての列を表示する設定（変更しないでください）
pd.options.display.max_columns = None

## 2. データの読み込み

seabornに添付のデータセットから「titanic」を読み込み、内容を確認します。

In [2]:
# seabornからtitanicのデータセットを読み込む（変更しないでください）
dataset = sns.load_dataset("titanic")

`sns.load_dataset()` で読み込んだデータは、pandasのDataFrameになっています。

In [3]:
# datasetの先頭5件を確認
print(dataset.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


### 使用する列の指定

今回は `survived, pclass, sex, age, sibsp, parch, fare, embarked` の列を使用します。

#### 参考:各列の説明

- `survived`: 生存区分（0:死亡, 1:生存）
- `pclass`: チケットクラス
- `sex`: 性別（male:男性, female:女性）
- `age`: 年齢
- `sibsp`: 同乗している兄弟や配偶者の数
- `parch`: 同乗している親や子供の数
- `fare`: 料金
- `embarked`: 乗船した港（頭文字）
- `class`: 客室クラス
- `who`: 性別（man:男性, woman:女性）
- `adult_male`: 成人男性ならTrue
- `deck`: 事故の際にどのデッキにいたか
- `embark_town`: 乗船した港名
- `alive`: 生存区分（no:死亡, yes:生存）
- `alone`: 1人で乗船したか

In [4]:
# datasetから「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を取得して
# datasetに代入（上書き）する
dataset = dataset[["survived", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]]

In [5]:
# 改めてdatasetの先頭5件を表示
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## 3. データの前処理

### 要約統計量の表示

In [6]:
# 要約統計量を表示
dataset.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 欠損値の確認と補完

In [7]:
# 各列の欠損値の数を確認
dataset.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

ageの欠損値は平均値で補完します。

In [8]:
# ageの欠損値を、ageの平均値で補完する
dataset['age'] = dataset['age'].fillna(dataset['age'].mean())

# 確認のため、再度欠損値の数を確認
print(dataset.isnull().sum())

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    2
dtype: int64


embarkedの欠損値は、もっとも乗船者数の多い港で補完します。

その方法はいくつかありますが、ここではその1つとして、DataFrameの特定の1列（Series）が持つ `value_counts()` メソッドを紹介します。このメソッドを実行すると、その列が持つ値ごとのデータ数がわかります。

参考：[pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

In [9]:
# 乗船者数の多い港を value_counts メソッドで確認
dataset['embarked'].value_counts()

embarked
S    644
C    168
Q     77
Name: count, dtype: int64

`values_count()` の結果を見て、もっとも乗船者数の多い港の文字で欠損値を埋めるようにします。

In [11]:
# embarkedの欠損値をもっとも乗船者数の多い港にて補完
# 'S' で欠損値を補完
dataset['embarked'] = dataset['embarked'].fillna('S')

上記の処理により、欠損値がなくなったかを確認しましょう。

In [12]:
# 欠損値の数を確認し、補完後の欠損値が0であることを確認
# 確認のために欠損値の合計を再表示
dataset.isnull().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

### ダミー変数への変換

sexとembarkedをダミー変数に変換します。

In [13]:
# datasetのsexとembarkedをダミー変数に変換してdataset2に代入する
dataset2 = pd.get_dummies(dataset, columns=['sex', 'embarked'])

In [14]:
# dataset2のデータの最初の5行を表示
dataset2.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,0,3,22.0,1,0,7.2500,False,True,False,False,True
1,1,1,38.0,1,0,71.2833,True,False,True,False,False
2,1,3,26.0,0,0,7.9250,True,False,False,False,True
3,1,1,35.0,1,0,53.1000,True,False,False,False,True
4,0,3,35.0,0,0,8.0500,False,True,False,False,True


## 4. 目的変数と説明変数の選択

ここでは、以下の列を使用します。

- 目的変数: `survived`
- 説明変数: それ以外

dataset2より目的変数と説明変数に該当する列を取得してnumpy配列に変換し、変数YとXに格納します。列の除外には、DataFrameの `drop` を使います。`データフレーム.drop(columns=除外したい列名)` です。

In [15]:
# Y:目的変数に該当する列
Y = dataset2['survived'].values

# X:説明変数に該当する列。dataset2からsurvivedを除外
X = dataset2.drop(columns='survived').values

In [16]:
# YとXの形状を確認
print("X（説明変数）の形状:", X.shape)
print("Y（目的変数）の形状:", Y.shape)

X（説明変数）の形状: (891, 10)
Y（目的変数）の形状: (891,)


## 5. データの分割

この課題ではホールドアウト法でデータを分割します。

In [17]:
# X と Y を 機械学習用データとテストデータに7:3に分ける(X_train, X_test, Y_train, Y_test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [18]:
# 機械学習用データを、学習データと検証データに7:3に分ける(X_train, X_valid, Y_train, Y_valid)
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, random_state=0)

In [19]:
# 形状を確認:X_train, X_valid, X_test, Y_train, Y_valid, Y_test
# 各データの形状（Shape）を確認
print("--- 説明変数 (X) ---")
print(f"X_train: {X_train.shape}")
print(f"X_valid: {X_valid.shape}")
# もしX_testも作成済みの場合は以下も表示
# print(f"X_test : {X_test.shape}")

print("\n--- 目的変数 (Y) ---")
print(f"Y_train: {Y_train.shape}")
print(f"Y_valid: {Y_valid.shape}")
# print(f"Y_test : {Y_test.shape}")

--- 説明変数 (X) ---
X_train: (623, 10)
X_valid: (268, 10)

--- 目的変数 (Y) ---
Y_train: (623,)
Y_valid: (268,)


## 6. モデルの選択

ロジスティック回帰と決定木、ランダムフォレスト、SVMの4つのモデルを作成して比較します。

In [20]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

モデルの評価（性能の比較）には、F1値を使ってください。以下には1つだけセルを用意していますが、モデルを4つ作って比較する処理のためにセルを増やしてもかまいません。

In [21]:
# 4つのモデルを作成し、それぞれのF1値を出力する
# モデルをリストに格納
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=0),
    'Random Forest': RandomForestClassifier(random_state=0),
    'SVC': SVC()
}

# 各モデルで学習と評価を実行
for name, model in models.items():
    # 学習
    model.fit(X_train, Y_train)
    
    # 予測
    Y_pred = model.predict(X_valid)
    
    # F1スコアの算出
    score = f1_score(Y_valid, Y_pred)
    
    print(f"{name:20}: F1 Score = {score:.4f}")

Logistic Regression : F1 Score = 0.7236
Decision Tree       : F1 Score = 0.6965
Random Forest       : F1 Score = 0.7526
SVC                 : F1 Score = 0.4571


## 7. パラメータのチューニング

GridSearchCVを使い、性能の良かったランダムフォレストのパラメータのチューニングを行ないます。パラメータの候補については、レッスン本編を参考にしてください。

In [22]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.model_selection import GridSearchCV

In [23]:
# 性能の良かったモデルを作成
# 最も性能が良かった RandomForest を作成（設定はデフォルト、または微調整）
best_model = RandomForestClassifier(n_estimators=100, random_state=0)

# 学習データ全体を使って学習
best_model.fit(X_train, Y_train)

# 検証データで最終確認
Y_pred = best_model.predict(X_valid)
final_score = f1_score(Y_valid, Y_pred)

print(f"採用モデル: Random Forest")
print(f"検証データでのF1値: {final_score:.4f}")

採用モデル: Random Forest
検証データでのF1値: 0.7526


In [24]:
# パラメータの指定
# 探索するパラメーターの範囲を指定
param_grid = {
    'n_estimators': [50, 100, 200, 300], # 作成する木の数
    'max_depth': [None, 3, 5, 7, 10],      # 木の深さ（深すぎると過学習の原因に）
    'min_samples_split': [2, 5, 10],      # 枝分かれするために必要な最小サンプル数
    'criterion': ['gini', 'entropy']      # 分割の品質を測る指標
}

print("探索パラメーターを定義しました。")

探索パラメーターを定義しました。


In [25]:
# グリッドサーチのオブジェクトを作成
# GridSearchCVのオブジェクトを作成
# estimator: モデル（RandomForestClassifier）
# param_grid: 先ほど定義したパラメーター候補
# cv: データを5つに分けて検証する（5-fold Cross Validation）
# scoring: 評価指標にf1スコアを指定
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=0),
    param_grid=param_grid,
    cv=5,
    scoring='f1'
)

print("GridSearchCVオブジェクトの作成が完了しました。")

GridSearchCVオブジェクトの作成が完了しました。


In [26]:
# データの分割:機械学習用データを学習と検証に分けるのはクロスバリデーションで行ってくれる
# （Xg_train, Xg_test, Yg_train, Yg_test）
# 全体の30%を最終テスト用(test)に、70%をグリッドサーチ用(train)に分割
Xg_train, Xg_test, Yg_train, Yg_test = train_test_split(X, Y, test_size=0.3, random_state=0)

print(f"グリッドサーチ用(Xg_train): {Xg_train.shape}")
print(f"最終テスト用(Xg_test): {Xg_test.shape}")

グリッドサーチ用(Xg_train): (623, 10)
最終テスト用(Xg_test): (268, 10)


In [27]:
# グリッドサーチを実行する
grid_search.fit(Xg_train, Yg_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestC...andom_state=0)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'criterion': ['gini', 'entropy'], 'max_depth': [None, 3, ...], 'min_samples_split': [2, 5, ...], 'n_estimators': [50, 100, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'f1'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and p

In [28]:
# 最適なパラメータを表示
# 最適なパラメーターを表示
print("Best Parameters:")
print(grid_search.best_params_)

# その時の交差検証での平均F1値
print(f"\nBest Cross-Validation F1 Score: {grid_search.best_score_:.4f}")

Best Parameters:
{'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 10, 'n_estimators': 50}

Best Cross-Validation F1 Score: 0.7610


ここで得たパラメータをもとに、モデルを再度作成します。

In [29]:
# 最適なパラメータによるモデルの作成
best_rf_model = grid_search.best_estimator_

# モデルの学習
best_rf_model.fit(Xg_train, Yg_train)

# モデルの予測
Yg_pred = best_rf_model.predict(Xg_test)

In [30]:
# F1値の出力
final_f1_score = f1_score(Yg_test, Yg_pred)

## 8. テストデータによる汎化性能の確認

最後にテストデータでモデルの汎化性能を確認しましょう。

In [31]:
# テストデータを使って予測を行いF1値を算出
Yg_pred = best_rf_model.predict(Xg_test)

# F1値を算出
final_f1 = f1_score(Yg_test, Yg_pred)

print(f"最終テストデータに対するF1値: {final_f1:.4f}")

最終テストデータに対するF1値: 0.7653
